## Baseline model from tensorflow tutorial
#### trained on old data: 76k of 3 channels images
#### conv1: 32 features, 5 x 5, max pool 2, 2
#### conv2: 64 features, 5 x 5, max pool 2, 2
#### fc1: 1024
#### no dropout

In [1]:
import gen_input
import numpy as np
import os
from datetime import datetime
from time import time
import tensorflow as tf

In [2]:
train_test_valid_split = [1., 0., 0.]
svhn_train = gen_input.read_data_sets("data/train_32x32.mat", train_test_valid_split).train
svhn_test = gen_input.read_data_sets("data/test_32x32.mat", train_test_valid_split).train

print svhn_train.images[0].shape
print svhn_train.images.shape
print svhn_test.images[0].shape
print svhn_test.images.shape

(1024, 3)
(73257, 1024, 3)
(1024, 3)
(26032, 1024, 3)


## Model Wrappers

In [3]:
# Conv2D wrapper, with bias and relu activation
def conv2d(x, W, b, strides=1):
  x = tf.nn.conv2d(x, W, 
                   strides=[1, strides, strides, 1], 
                   padding='SAME')
  x = tf.nn.bias_add(x, b)
  return tf.nn.relu(x)

# MaxPool2D wrapper
def maxpool2d(x, k=2):
  return tf.nn.max_pool(x, 
                        ksize=[1, k, k, 1], 
                        strides=[1, k, k, 1],
                        padding='SAME')

# 2-layer Convolutional network with 2x2 max pooling
def conv_net(x, weights, biases, keep_prob):
  # Reshape input
  x = tf.reshape(x, shape=[-1, 32, 32, 3])

  conv1 = conv2d(x, weights['wc1'], biases['bc1'])
  conv1 = maxpool2d(conv1, k=2)

  conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
  conv2 = maxpool2d(conv2, k=2)

  # Reshape conv2 output to fit fully connected layer input
  fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
  # Fully connected layer
  fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
  fc1 = tf.nn.relu(fc1)
  fc1 = tf.nn.dropout(fc1, keep_prob)

  # Output classes
  out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
  return out

In [20]:
def run():
  # RESET TF GRAPH, just in case
  tf.reset_default_graph()

  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 1024, 3], name="x_input") 
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y_actual")
    tf.add_to_collection('x', x)
    tf.add_to_collection('y_', y_)

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.add_to_collection('keep_prob', keep_prob)

  with tf.name_scope('weights'):
    weights = {
        'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32]), name="weights_conv1"),
        'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64]), name="weights_conv2"),
        'wd1': tf.Variable(tf.random_normal([8*8*64, 1024]), name="weights_fc1"),
        'out': tf.Variable(tf.random_normal([1024, 10]), name="weights_output")
    }

  with tf.name_scope('biases'):
    biases = {
        'bc1': tf.Variable(tf.constant(0.0, shape=[32]), name="bias_conv1"),
        'bc2': tf.Variable(tf.constant(0.0, shape=[64]), name="bias_conv2"),
        'bd1': tf.Variable(tf.constant(0.0, shape=[1024]), name="bias_fc1"),
        'out': tf.Variable(tf.constant(0.0, shape=[10]), name="bias_output")
    }

  with tf.name_scope('Model'):
    logits = conv_net(x, weights, biases, keep_prob)
    tf.add_to_collection('ops', logits)

  with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))

  with tf.name_scope('Accuracy'):
    accuracy = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(accuracy, tf.float32))

  with tf.name_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


  # Initializing the variables
  init = tf.global_variables_initializer()

  # Summaries to visualize loss & accuracy
  tf.summary.scalar("loss", loss)
  tf.summary.scalar("accuracy", accuracy)

  merged_summaries = tf.summary.merge_all()
  
  # for saving the model in the end
  saver = tf.train.Saver()

  ts = datetime.now().strftime('%Y%m%d_%H%M')
  logs_path = "logs/{}/".format(ts)
  print "-"* 70
  pwd = os.getcwd()+"/"
  print("Run the following to start tensorboard server:\n" \
        "tensorboard --logdir=/{}{}".format(pwd, logs_path))

  ##########################################
  ##                                      ##
  ##           Launch the graph           ##
  ##                                      ##
  ##########################################
  
  # Normalize by subtracting per image, per channel means
  def normalize_batch(batch):
    per_img_ch_means = batch.mean(axis=1)
    return batch - per_img_ch_means[:, np.newaxis, :]

  # Fill in the place holders depending on the context (training? testing?)
  def feed_dict(mode):
    if mode == 'train':
      batch_x, batch_y = svhn_train.next_batch(batch_size)
      keep_proba = train_keep_prob

    elif mode == 'train_no_dropout':
      batch_x, batch_y = svhn_train.next_batch(test_batch_size)
      keep_proba = 1.0

    elif mode == 'test_no_dropout':
      batch_x, batch_y = svhn_test.next_batch(test_batch_size)
      keep_proba = 1.0

    batch_x = normalize_batch(batch_x) # Subtract per image mean
    return {x: batch_x, y_: batch_y, keep_prob: keep_proba} # can't name values same as keys

  with tf.Session() as sess:
    t_start = time()
    sess.run(init)
    train_writer = tf.summary.FileWriter(logs_path + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(logs_path + '/test')
    # Training loop
    for epoch in xrange(training_epochs):
      for batch_num in xrange(total_batches):
        if batch_num % test_every == test_every - 1:

          # Record summaries and accuracy on the *test* set
          summary, acc = sess.run([merged_summaries, accuracy], feed_dict=feed_dict(mode='test_no_dropout'))
          test_writer.add_summary(summary, epoch * total_batches + batch_num)

          # To compare against *training* set (apples to apples comparison)
          summary, _ = sess.run([merged_summaries, optimizer], feed_dict=feed_dict(mode='train_no_dropout'))
          train_writer.add_summary(summary, epoch * total_batches + batch_num)

          # Print occasional progress
          print('Test accuracy at epoch %s: batch %s: %s' % (epoch, batch_num, acc))
        else:
          sess.run([optimizer], feed_dict=feed_dict(mode='train'))
    
    t_end = time()
    elapsed_mins = (t_end - t_start) / 60.0
    print "\nOptimization Finished! in {} minutes".format(elapsed_mins)
    
    # Save down the current model
    if not os.path.exists("models"): os.makedirs("models")
    saver.save(sess, "models/{}".format(model_name))

In [21]:
##########################################
##                                      ##
##              Parameters              ##
##                                      ##
##########################################

# Training Parameters
learning_rate = 0.001
training_epochs = 3

batch_size = 100
test_batch_size = 5000 #50
test_every = 100

total_batches = int(svhn_train.num_examples / batch_size)

# Drop out 
train_keep_prob = 1.0

model_name = "baseline_old_data"

run()

----------------------------------------------------------------------
Run the following to start tensorboard server:
tensorboard --logdir=//Users/baha/workspace/svhn/logs/20170423_1704/

Optimization Finished! in 0.0617247819901 minutes


## Testing inference

In [31]:
saver = tf.train.Saver()
x_batch, y_batch = svhn_test.next_batch(10)

with tf.Session() as sess:
  saver.restore(sess, "models/{}".format(model_name))
  print "model restored"
  #logits

  logits = tf.get_collection("logits")[0]
  x = tf.get_collection("x")[0]
  y_ = tf.get_collection("y_")[0]
  keep_prob = tf.get_collection("keep_prob")[0]
  
  predictions = sess.run(logits, feed_dict={x: x_batch, y_: y_batch, keep_prob: 1.0})
  y_preds = tf.argmax(predictions, 1)
  y_actual = tf.argmax(y_batch, 1)
  
  ## tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
  print sess.run(y_preds)
  print sess.run(y_actual)

model restored
[<tf.Tensor 'input/x_input:0' shape=(?, 1024, 3) dtype=float32>]
[4 4 4 4 0 2 2 2 2 2]
[3 7 6 3 0 0 2 7 4 2]
